In [57]:
import pandas as pd

df = pd.read_csv("../../data/preprocess/pre-processed.csv")
df.drop(columns=["Profit", "Selling", "Revenue"], inplace=True)
df.dtypes

Field Name        object
Crop ID            int64
Crop Name         object
Crop Type         object
Planting Area    float64
Season            object
Field Type        object
Field Area       float64
Field ID           int64
Per Yield        float64
Per Cost         float64
Price            float64
Yield            float64
Cost             float64
dtype: object

# 符号定义

- $S_{jk}$：第 $k$ 年第 $j$ 作物的期望销量
- $Y_{jk}$：第 $k$ 年第 $j$ 作物的单位面积产量
- $P_{jk}$：第 $k$ 年第 $j$ 作物的售价
- $C_{jk}$：第 $k$ 年第 $j$ 作物的单位面积成本
- $A_{i}^*$： 第 $i$ 个地块的面积
- $A^1_{ijks}$：第 $k$ 年第 $s$ 季度的第 $i$ 大棚上，第 $j$ 作物的种植面积
- $A^2_{ijks}$：第 $k$ 年第 $s$ 季度的第 $i$ 非大棚地块上，第 $j$ 作物的种植面积
- $A_{ijk}$：第 $k$ 年第 $i$ 地块上，第 $j$ 作物的总种植面积，定义为： $A_{ijk} = \sum_s A^2_{ijks} + A^1_{ij(k-1)2} + A^1_{ijks}$

- $t_i$： 第$i$个地块的类型（如平旱地、梯田、山坡地、智能大棚、普通大棚、水浇地）
- $\hat{T}_{is}$：第 $i$ 个地块在第 $s$ 季可种植的作物集合
- $\text{Beans}$：豆类作物的集合
- $\text{Grains}_A$：A类粮食作物的集合，即除了水稻之外的粮食作物
- $\text{Grains}_B$：B类粮食作物的集合，即水稻
- $\text{Vege}_A$：A类蔬菜的集合，即除了大白菜、白萝卜和红萝卜之外的蔬菜
- $\text{Vege}_B$：B类蔬菜的集合，即大白菜、白萝卜和红萝卜
- $\text{Mush}$：食用菌的集合


# 决策变量

- $A^1_{ijks}$：第 $k$ 年第 $s$ 季度的第 $i$ 大棚上，第 $j$ 作物的种植面积
- $A^2_{ijks}$：第 $k$ 年第 $s$ 季度的第 $i$ 非大棚地块上，第 $j$ 作物的种植面积

# 约束条件
- 每种作物合起来的种植面积不超过相应地块的总面积
    $$
    \sum_j A_{ijks} \leq A_i^* \quad \forall i,k \text{ and } j \in \hat{T}_{is}
    $$

- 每种作物在同一地块（含大棚）都不能连续重茬种植
    $$
    A_{ij(k-1)s}+A_{ijks} \leq min(A_{ij(k-1)s},A_{ijks}) \quad \forall i,k \text{ and } j \in \hat{T}_{is}
    $$

- 每个地块（含大棚）的所有土地三年内至少种植一次豆类作物
    $$
    max(A_{ij(k-2)s}, A_{ij(k-1)s},A_{ijks}) = A_i^* \quad \forall i,k \text{ and } j \in \hat{T}_{is}
    $$

- 每种作物在单个地块（含大棚）种植的面积不宜太小
    $$
    A_{ijks}^{n} \geq M \times A_i^*  \quad \text{if } A_{ijks}^{n} \neq 0 \qquad \forall i,k \text{ and } j \in \hat{T}_{is},n \in \{1,2\}
    $$

- 种植的地块类型限制和季节限制
    $$
    \hat{T}_i =
    \begin{cases}
    \begin{aligned}
        & \hat{T}_{i1} = \text{Grains}_A, \quad \hat{T}_{i2} = \phi, \quad & \text{if } t_i \in \{\text{平旱地}, \text{梯田}, \text{山坡地}\} \\
    \end{aligned} \\
    \begin{aligned}
        & \hat{T}_{i1} = \text{Grains}_B \quad \text{或} \quad \hat{T}_{i1} = \text{Veg}_A, \quad \hat{T}_{i2} = \text{Vege}_B, \quad & \text{if } t_i \in \{\text{水浇地}\} \\
    \end{aligned} \\
    \begin{aligned}
        & \hat{T}_{i1} = \text{Vege}_A, \quad \hat{T}_{i2} = \text{Mush}, \quad & \text{if } t_i \in \{\text{普通大棚}\} \\
    \end{aligned} \\
    \begin{aligned}
        & \hat{T}_{i1} = \hat{T}_{i2} = \text{Vege}_A, \quad & \text{if } t_i \in \{\text{智慧大棚}\} \\
    \end{aligned} \\
    \end{cases}
    $$


# 情况一

如果某种作物的总产量超过预期销售量，超出部分滞销

$$
L_1 = \sum_{ijk} \left( \min \left( S_{jk}, Y_{jk} \cdot A_{ijk} \right) \cdot P_{jk} - C_{jk} \cdot A_{ijk} \right)
$$

In [76]:
import pulp
import numpy as np

# 创建一个线性规划问题
prob = pulp.LpProblem("Crop_Optimization", pulp.LpMaximize)

# 获取辅助变量

fields = df["Field ID"].unique()  # 所有的地块
crops = df["Crop ID"].unique()  # 所有的作物
seasons = df["Season"].unique()  # 所有的时节
years = range(2024, 2031)  # 要优化的年份


def pop_ndarray(arr, element_to_pop):
    index = np.where(arr == element_to_pop)[0]
    if index.size > 0:
        arr = np.delete(arr, index)
    return arr


# A类粮食作物的集合
grains_A = df[df["Crop Type"] == "粮食"]["Crop ID"].unique()
grains_A = pop_ndarray(grains_A, 16)

# B类粮食作物的集合
grains_B = np.array([16])

# A类蔬菜作物的集合
vege_A = df[df["Crop Type"] == "蔬菜"]["Crop ID"].unique()
for i in range(3):
    vege_A = pop_ndarray(vege_A, 35 + i)

# B类蔬菜作物的集合
vege_B = np.array([35, 36, 37])

# 食用菌作物的集合
mush = df[df["Crop Type"] == "食用菌"]["Crop ID"].unique()

# 豆类作物的集合
beans = df[(df["Crop Type"] == "粮食（豆类）") | (df["Crop Type"] == "蔬菜（豆类）")][
    "Crop ID"
].unique()

# 第 i 个地块的类型（如平旱地、梯田、山坡地、智能大棚、普通大棚、水浇地）
t_i = {i: df[df["Field ID"] == i]["Field Type"].iloc[0] for i in fields}

# 第 i 个地块在第 s 季可种植的作物集合
T_hat_i_s = {}
water_field_first_season_choice = {}  # 记录水浇地在第一季的选择


# 随机获取水浇地在第一季的选择
def get_first_season_choice():
    import random

    return random.choice(list(np.concatenate([grains_B, vege_A])))


for i in fields:
    if t_i[i] in ["平旱地", "梯田", "山坡地"]:
        T_hat_i_s[(i, "第一季")] = grains_A
        T_hat_i_s[(i, "第二季")] = None
    elif t_i[i] in ["水浇地"]:
        # 假设我们有一个函数 get_first_season_choice(i) 来获取水浇地在第一季的选择
        first_season_choice = get_first_season_choice()
        water_field_first_season_choice[i] = first_season_choice
        if first_season_choice in grains_B:
            T_hat_i_s[(i, "第一季")] = grains_B
            T_hat_i_s[(i, "第二季")] = None
        elif first_season_choice in vege_A:
            T_hat_i_s[(i, "第一季")] = vege_A
            T_hat_i_s[(i, "第二季")] = vege_B
    elif t_i[i] in ["普通大棚"]:
        T_hat_i_s[(i, "第一季")] = vege_A
        T_hat_i_s[(i, "第二季")] = mush
    elif t_i[i] in ["智能大棚"]:
        T_hat_i_s[(i, "第一季")] = vege_A
        T_hat_i_s[(i, "第二季")] = vege_A

S = {}  # 期望产量
Y = {}  # 单位面积产量
P = {}  # 售价
C = {}  # 单位面积成本
A_star = {}  # 地块面积

for i in fields:
    A_star[i] = df[df["Field ID"] == i]["Field Area"].iloc[0]

for j in crops:
    # 所有参数与 2023 年的保持一致
    for k in years:
        S[(j, k)] = df.groupby(["Crop ID"])["Yield"].agg("sum")[j]
        Y[(j, k)] = df[df["Crop ID"] == j]["Per Yield"].iloc[0]
        P[(j, k)] = df[df["Crop ID"] == j]["Price"].iloc[0]
        C[(j, k)] = df[df["Crop ID"] == j]["Per Cost"].iloc[0]

# 创建决策变量
A1 = pulp.LpVariable.dicts(
    "A1",
    [(i, j, k, s) for i in fields for j in crops for k in years for s in seasons],
    lowBound=0,
    cat="Continuous",
)
A2 = pulp.LpVariable.dicts(
    "A2",
    [(i, j, k, s) for i in fields for j in crops for k in years for s in seasons],
    lowBound=0,
    cat="Continuous",
)


# Loosely check inequality and not abs function used
def min_loose(a, b):
    return a if a <= b else b


# Loosely check inequality and not abs function used
def max_loose(a, b):
    return a if a >= b else b


# 定义目标函数

# A_{ijk} = \sum_s A^2_{ijks} + A^1_{ij(k-1)2} + A^1_{ijks}
A = {}
for i in fields:
    for j in crops:
        for k in years:
            A[(i, j, k)] = (
                pulp.lpSum([A2[(i, j, k, s)] for s in seasons])
                + (A1[(i, j, k - 1, "第二季")] if (i, j, k - 1, "第二季") in A1 else 0)
                + pulp.lpSum([A1[(i, j, k, s)] for s in seasons])
            )

# L_1 = \sum_{ijk} \left( \min \left( S_{jk}, Y_{jk} \cdot A_{ijk} \right) \cdot P_{jk} - C_{jk} \cdot A_{ijk} \right)
L1 = pulp.lpSum(
    [
        (
            min_loose(S[(j, k)], Y[(j, k)] * A[(i, j, k)]) * P[(j, k)]
            - C[(j, k)] * A[(i, j, k)]
        )
        for i in fields
        for j in crops
        for k in years
    ]
)

# L_2 = \sum_{ijk} \left( Y_{jk} \cdot A_{ijk} \cdot P_{jk} - C_{jk} \cdot A_{ijk} - 0.5 \cdot \max \left( 0, Y_{jk} \cdot A_{ijk} - S_{jk} \right) \cdot P_{jk} \right)
L2 = pulp.lpSum(
    [
        (
            Y[(j, k)] * A[(i, j, k)] * P[(j, k)]
            - C[(j, k)] * A[(i, j, k)]
            - 0.5 * max_loose(0, Y[(j, k)] * A[(i, j, k)] - S[(j, k)]) * P[(j, k)]
        )
        for i in fields
        for j in crops
        for k in years
    ]
)

# 设置目标函数
prob += L1 + L2

# 约束条件一：\sum_j A_{ijks} \leq A_i^* \quad \forall i,k \text{ and } j \in \hat{T}_{is}
for i in fields:
    for k in years:
        for s in seasons:
            if T_hat_i_s[(i, s)] is not None:
                prob += (
                    pulp.lpSum([A1[(i, j, k, s)] for j in T_hat_i_s[(i, s)]])
                    + pulp.lpSum([A2[(i, j, k, s)] for j in T_hat_i_s[(i, s)]])
                    <= A_star[i]
                )

# 约束条件二：A_{ij(k-1)s}+A_{ijks} \leq min(A_{ij(k-1)s},A_{ijks}) \quad \forall i,k \text{ and } j \in \hat{T}_{is}
for i in fields:
    for k in years:
        for s in seasons:
            if T_hat_i_s[(i, s)] is not None:
                prob += A1[(i, j, k - 1, s)] + A1[(i, j, k, s)] <= min_loose(
                    A1[(i, j, k - 1, s)], A1[(i, j, k, s)]
                )
                prob += A2[(i, j, k - 1, s)] + A2[(i, j, k, s)] <= min_loose(
                    A2[(i, j, k - 1, s)], A2[(i, j, k, s)]
                )

# 约束条件三：A_{ij(k-1)2} \leq A_{ij(k-1)1} \quad \forall i,k \text{ and } j \in \hat{T}_{is}
for i in fields:
    for k in years:
        prob += (
            A1[(i, j, k - 1, "第二季")] <= A1[(i, j, k - 1, "第一季")]
            for j in crops
            if (i, j, k - 1, "第一季") in A1
        )
        prob += (
            A2[(i, j, k - 1, "第二季")] <= A2[(i, j, k - 1, "第一季")]
            for j in crops
            if (i, j, k - 1, "第一季") in A2
        )

# 约束条件四：A_{ijks}^{n} \geq M \times A_i^*  \quad \text{if } A_{ijks}^{n} \neq 0 \qquad \forall i,k \text{ and } j \in \hat{T}_{is},n \in \{1,2\}
for i in fields:
    for k in years:
        for s in seasons:
            if T_hat_i_s[(i, s)] is not None:
                prob += (
                    A1[(i, j, k, s)] >= 0.1 * A_star[i]
                    for j in T_hat_i_s[(i, s)]
                    if (i, j, k, s) in A1
                )
                prob += (
                    A2[(i, j, k, s)] >= 0.1 * A_star[i]
                    for j in T_hat_i_s[(i, s)]
                    if (i, j, k, s) in A2
                )

# 解决问题
prob.solve()

# 输出结果
for v in prob.variables():
    print(v.name, "=", v.varValue)

terminate called after throwing an instance of 'std::bad_alloc'
  what():  std::bad_alloc


PulpSolverError: Pulp: Error while trying to execute, use msg=True for more details/home/zivmax/CUMCM_2024/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc